# **Gesture Recognition**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import cv2
import os
from google.colab.patches import cv2_imshow
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from keras.models import Sequential
from keras.applications.vgg16 import VGG16
from keras.layers import Dense, InputLayer, Dropout, Flatten
from sklearn.utils import shuffle
import random
import datetime as dt
from moviepy.editor import *
from collections import deque
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import plot_model

Imageio: 'ffmpeg-linux64-v3.3.1' was not found on your computer; downloading it now.
Try 1. Download from https://github.com/imageio/imageio-binaries/raw/master/ffmpeg/ffmpeg-linux64-v3.3.1 (43.8 MB)
Downloading: 8192/45929032 bytes (0.0%)2924544/45929032 bytes (6.4%)7184384/45929032 bytes (15.6%)11206656/45929032 bytes (24.4%)15360000/45929032 bytes (33.4%)19546112/45929032 bytes (42.6%)23740416/45929032 bytes (51.7%)27656192/45929032 bytes (60.2%)31875072/45929032 bytes (69.4%)35921920/45929032 bytes (78.2%)39976960/45929032 bytes (87.0%)44179456/45929032 bytes (96.2%)45929032/45929032 bytes (100.0%)
  Done
File saved as /root

In [ ]:
dir_videos = '/content/drive/MyDrive/AI-project/Gesture-Recognition/dataset/videos'
dir_merged = '/content/drive/MyDrive/AI-project/Gesture-Recognition/merged'
#dir_images = '/content/drive/MyDrive/AI-project/Gesture-Recognition/dataset/images'
#dir_augumented_images = '/content/drive/MyDrive/AI-project/Gesture-Recognition/dataset/augumented-images'

#Extract frames from videos and dataset creation

##Initilize

In [ ]:
IMAGE_HEIGHT, IMAGE_WIDTH = 224, 224
IMAGE_SIZE = (224, 224, 3)

classes_list = ['bandaging', 'bowling', 'breakdancing', 'ironing', 'kissing', 'riding_scooter', 'side_kick', 'tap_dancing', 'texting', 'washing_hair']
print(f'The number of classes: {len(classes_list)}')

max_images_per_class = 1000
max_images_per_video = 10

The number of classes: 10


In [ ]:
seed_constant = 23
np.random.seed(seed_constant)
random.seed(seed_constant)
tf.random.set_seed(seed_constant)

##Function to extract, resize and normalize frames

In [ ]:
# import math
# def frames_extraction(pathFilename):
#   frames_list = []
#   cap = cv2.VideoCapture(pathFilename)
#   #total_frames = cap.get(cv2.CAP_PROP_FRAME_COUNT)
#   #skip_frames_window = max(int(total_frames/SEQUENCE_LENGTH), 1)
  
#   #for count in range(SEQUENCE_LENGTH):
#   count_frames = 0

#   while True:
#     #cap.set(cv2.CAP_PROP_POS_FRAMES, count * skip_frames_window)
#     success, frame = cap.read()
#     if not success:
#       break
    
#     count_frames += 1
#     if count_frames % 5 == 0:
#       resized_frame = cv2.resize(frame, (IMAGE_HEIGHT, IMAGE_WIDTH))
#       normalized_frame = resized_frame / 255.
#       frames_list.append(normalized_frame)

#   cap.release()
#   return frames_list

In [ ]:
import math

def frames_extraction(pathFilename):
  frames_list = []
  cap = cv2.VideoCapture(pathFilename)
  
  while True:
    success, frame = cap.read()
    if not success:
      break

    resized_frame = cv2.resize(frame, (IMAGE_HEIGHT, IMAGE_WIDTH))
    normalized_frame = resized_frame / 255.
    frames_list.append(normalized_frame)

  cap.release()
  return frames_list

#Create and train model

##Create model

In [ ]:
densenet_model = Sequential()

pretrained_model= tf.keras.applications.DenseNet201(include_top=False,
                   input_shape=(IMAGE_HEIGHT, IMAGE_WIDTH, 3),
                   pooling='avg',
                   weights='imagenet')
for layer in pretrained_model.layers:
        layer.trainable=False

densenet_model.add(pretrained_model)

74850304/74836368 [==============================] - 1s 0us/step


In [ ]:
densenet_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 densenet201 (Functional)    (None, 1920)              18321984  
                                                                 
Total params: 18,321,984
Trainable params: 0
Non-trainable params: 18,321,984
_________________________________________________________________


In [ ]:
resnet_model = Sequential()

pretrained_model= tf.keras.applications.ResNet50(include_top=False,
                   input_shape=(IMAGE_HEIGHT, IMAGE_WIDTH, 3),
                   pooling='avg',
                   weights='imagenet')
for layer in pretrained_model.layers:
        layer.trainable=False

resnet_model.add(pretrained_model)

94781440/94765736 [==============================] - 1s 0us/step


In [ ]:
resnet_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 2048)              23587712  
                                                                 
Total params: 23,587,712
Trainable params: 0
Non-trainable params: 23,587,712
_________________________________________________________________


In [ ]:
video_features = []
labels = []
paths_list = []

In [ ]:
def video_to_resnet_feature(path_list, start, model=densenet_model, num_of_frames=64):
  number = 0
  for idx in range(start, len(path_list)):
    path = path_list[idx]
    number += 1
    print(f'The numberical order: {number}')
    print(path)
    frame_list = frames_extraction(path)
    size = len(frame_list)
    print(size)
    step = size // num_of_frames
    if step <= 2:
        video_features.append(None)
        print(path)
        continue
    chosen_frame = []
    for frame_index in range(0, size, step):
      chosen_frame.append(frame_list[frame_index])
    while len(chosen_frame) > num_of_frames:
      chosen_frame.pop()
    chosen_frame = tf.stack(chosen_frame)
    output_feature = model(chosen_frame)
    video_features.append(output_feature)

subsets_list = ['train', 'validate']

def create_video_dataset(subsets_list = subsets_list, num_of_train_videos_each_class=230):
  for class_index, class_name in enumerate(classes_list):

    files_list = []
    for subset in subsets_list:
      print(f'Extracting {subset} data of class: {class_name}')
      temp_files_list = os.listdir(dir_videos + '/' + subset + '/' + class_name)
      for file in temp_files_list:
        files_list.append(dir_videos + '/' + subset + '/' + class_name + '/' + file)
    
    files_list = random.sample(files_list, num_of_train_videos_each_class)

    for file in files_list:
        paths_list.append(file)
        labels.append(class_index)

def fix_dataset(video_features=video_features, labels=labels):
    labels_temp = [labels[i] for i in range(len(labels)) if video_features[i] != None]
    video_features_temp = [video_features[i] for i in range(len(video_features)) if video_features[i] != None]
    return video_features_temp, labels_temp

In [ ]:
create_video_dataset()

Extracting train data of class: bandaging
Extracting validate data of class: bandaging
Extracting train data of class: bowling
Extracting validate data of class: bowling
Extracting train data of class: breakdancing
Extracting validate data of class: breakdancing
Extracting train data of class: ironing
Extracting validate data of class: ironing
Extracting train data of class: kissing
Extracting validate data of class: kissing
Extracting train data of class: riding_scooter
Extracting validate data of class: riding_scooter
Extracting train data of class: side_kick
Extracting validate data of class: side_kick
Extracting train data of class: tap_dancing
Extracting validate data of class: tap_dancing
Extracting train data of class: texting
Extracting validate data of class: texting
Extracting train data of class: washing_hair
Extracting validate data of class: washing_hair


In [ ]:
print(paths_list[0])
print(len(paths_list))

/content/drive/MyDrive/AI-project/Gesture-Recognition/dataset/videos/train/bandaging/vidHvaU7W635to.mp4
2300


In [ ]:
video_to_resnet_feature(paths_list, 0)

Streaming output truncated to the last 5000 lines.
/content/drive/MyDrive/AI-project/Gesture-Recognition/dataset/videos/train/ironing/vidHeCgJSfwgno.mp4
300
The numberical order: 696
/content/drive/MyDrive/AI-project/Gesture-Recognition/dataset/videos/train/ironing/vidqzBcsFmGmpE.mp4
251
The numberical order: 697
/content/drive/MyDrive/AI-project/Gesture-Recognition/dataset/videos/train/ironing/vidNPL7a9CcfVA.mp4
299
The numberical order: 698
/content/drive/MyDrive/AI-project/Gesture-Recognition/dataset/videos/train/ironing/vid2cBGeBKn9mU.mp4
299
The numberical order: 699
/content/drive/MyDrive/AI-project/Gesture-Recognition/dataset/videos/train/ironing/vidDtD0PFMlfTQ.mp4
299
The numberical order: 700
/content/drive/MyDrive/AI-project/Gesture-Recognition/dataset/videos/train/ironing/vid-3SmhZu4ZWc.mp4
299
The numberical order: 701
/content/drive/MyDrive/AI-project/Gesture-Recognition/dataset/videos/validate/ironing/vidchIaTW7kPdw.mp4
300
The numberical order: 702
/content/drive/MyDrive

In [ ]:
print(len(video_features))

2300


In [ ]:
fail_videos = [i for i in range(len(video_features)) if video_features[i] == None]
print(len(fail_videos))
print(fail_videos)

293
[17, 80, 85, 90, 91, 101, 105, 120, 136, 137, 145, 148, 150, 170, 194, 196, 201, 212, 230, 233, 248, 253, 254, 263, 268, 269, 270, 276, 278, 281, 283, 285, 301, 304, 313, 316, 319, 327, 335, 336, 343, 344, 351, 354, 355, 356, 358, 378, 381, 388, 399, 403, 407, 414, 417, 423, 424, 425, 432, 433, 438, 442, 446, 451, 454, 463, 464, 469, 470, 471, 472, 482, 489, 490, 495, 498, 506, 507, 510, 515, 520, 532, 535, 540, 541, 545, 553, 556, 557, 565, 579, 586, 591, 597, 601, 604, 609, 614, 631, 642, 643, 647, 653, 659, 660, 661, 670, 682, 686, 690, 724, 764, 768, 783, 788, 810, 820, 878, 912, 936, 953, 957, 973, 976, 977, 989, 993, 1003, 1008, 1030, 1054, 1060, 1081, 1107, 1109, 1139, 1148, 1173, 1174, 1204, 1216, 1218, 1231, 1234, 1238, 1241, 1244, 1260, 1268, 1270, 1271, 1281, 1287, 1299, 1301, 1305, 1312, 1327, 1328, 1337, 1341, 1344, 1367, 1371, 1376, 1379, 1380, 1391, 1397, 1401, 1405, 1409, 1411, 1414, 1417, 1418, 1421, 1423, 1426, 1427, 1428, 1429, 1435, 1446, 1448, 1449, 1466, 1470,

In [ ]:
fixed_video_features, fixed_labels = fix_dataset()

In [ ]:
# np.save('/content/drive/MyDrive/AI-project/Gesture-Recognition/features/fixed_video_features_resnet64_2300_avg.npy', np.array(fixed_video_features))
# np.save('/content/drive/MyDrive/AI-project/Gesture-Recognition/features/fixed_labels_resnet64_2300_avg.npy', np.array(fixed_labels))
np.save('/content/drive/MyDrive/AI-project/Gesture-Recognition/features/fixed_video_features_densenet64_2300_avg.npy', np.array(video_features))
np.save('/content/drive/MyDrive/AI-project/Gesture-Recognition/features/fixed_labels_densenet64_2300_avg.npy', np.array(labels))

In [ ]:
# np.save('/content/drive/MyDrive/AI-project/Gesture-Recognition/features/fixed_video_features_densenet_2300.npy', np.array(fixed_video_features))
# np.save('/content/drive/MyDrive/AI-project/Gesture-Recognition/features/fixed_labels_densenet_2300.npy', np.array(fixed_labels))

NameError: ignored

In [ ]:
model = resnet_model

In [ ]:
x_train = video_features
y_train = labels

NameError: ignored

In [ ]:
x_train = np.load('/content/drive/MyDrive/AI-project/Gesture-Recognition/features/fixed_video_features_resnet64_2300_avg.npy', allow_pickle=True)
y_train = np.load('/content/drive/MyDrive/AI-project/Gesture-Recognition/features/fixed_labels_resnet64_2300_avg.npy', allow_pickle=True)

In [ ]:
x_train = np.load('/content/drive/MyDrive/AI-project/Gesture-Recognition/features/fixed_video_features_densenet_2300.npy')
y_train = np.load('/content/drive/MyDrive/AI-project/Gesture-Recognition/features/fixed_labels_densenet_2300.npy')

In [ ]:
print(y_train)
print(len(y_train))

[0 0 0 ... 9 9 9]
2300


In [ ]:
x_train, y_train = fix_dataset(x_train.tolist(), y_train.tolist())

In [ ]:
y_train = tf.one_hot(y_train, 10)
print(y_train)

tf.Tensor(
[[1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]], shape=(2007, 10), dtype=float32)


In [ ]:
x_train = np.asarray(x_train)

In [ ]:
print(x_train.shape)
print(y_train.shape)

(2007, 64, 2048)
(2007, 10)


In [ ]:
temp = list(zip(x_train, y_train))
random.shuffle(temp)
x_train, y_train = zip(*temp)

In [ ]:
print(len(x_train))
print(len(y_train))
x_train = np.asarray(x_train)
y_train = np.asarray(y_train)

2007
2007


In [ ]:
print(x_train.shape)
print(y_train.shape)

(2007, 64, 2048)
(2007, 10)


# Compile and train models

## DenseAdam

In [ ]:
# opt = tf.keras.optimizers.SGD(
#     learning_rate=0.01, momentum=0.01, nesterov=True
# )



In [ ]:
from keras.layers.normalization.batch_normalization import BatchNormalization
from tensorflow.keras.layers import *
opt = tf.keras.optimizers.Adam()
classification_model = Sequential([
                                   Flatten(),
                                   Dense(512, activation='relu'),
                                   Dense(256, activation='relu'),
                                   Dense(10),
                                   Softmax()])

classification_model.compile(loss = 'categorical_crossentropy', optimizer = opt, metrics = ["accuracy"])


In [ ]:
classification_model_training_history = classification_model.fit(x = x_train, 
                                                                 y = y_train, 
                                                                 epochs = 1000, 
                                                                 batch_size = 512, 
                                                                 shuffle = True,
                                                                 validation_split = 0.2,
                                                                 )

Epoch 1/1000
4/4 [==============================] - 4s 333ms/step - loss: 16.2840 - accuracy: 0.1090 - val_loss: 34.7414 - val_accuracy: 0.1194
Epoch 2/1000
4/4 [==============================] - 1s 155ms/step - loss: 34.0436 - accuracy: 0.1047 - val_loss: 28.6505 - val_accuracy: 0.1045
Epoch 3/1000
4/4 [==============================] - 1s 153ms/step - loss: 20.6185 - accuracy: 0.1003 - val_loss: 12.3393 - val_accuracy: 0.0945
Epoch 4/1000
4/4 [==============================] - 1s 157ms/step - loss: 11.2162 - accuracy: 0.0953 - val_loss: 8.8547 - val_accuracy: 0.1119
Epoch 5/1000
4/4 [==============================] - 1s 152ms/step - loss: 6.7627 - accuracy: 0.1146 - val_loss: 5.7467 - val_accuracy: 0.0945
Epoch 6/1000
4/4 [==============================] - 1s 149ms/step - loss: 4.9227 - accuracy: 0.1140 - val_loss: 3.2011 - val_accuracy: 0.1119
Epoch 7/1000
4/4 [==============================] - 1s 149ms/step - loss: 3.0831 - accuracy: 0.1171 - val_loss: 2.9651 - val_accuracy: 0.1493

KeyboardInterrupt: ignored

In [ ]:
classification_model.load_weights('/content/drive/MyDrive/AI-project/Gesture-Recognition/dense_adamclassification_model')

In [ ]:
classification_model.save_weights('/content/drive/MyDrive/AI-project/Gesture-Recognition/resnet_adamclassification_model')

## Dense SGD

In [ ]:
opt = tf.keras.optimizers.SGD()
from keras.layers.normalization.batch_normalization import BatchNormalization
from tensorflow.keras.layers import *
classification_model = Sequential([
                                   Flatten(),
                                   Dense(512, activation='relu'),
                                   Dense(256, activation='relu'),
                                   Dense(10),
                                   Softmax()])

classification_model.compile(loss = 'categorical_crossentropy', optimizer = opt, metrics = ["accuracy"])


In [ ]:
classification_model_training_history = classification_model.fit(x = x_train, 
                                                                 y = y_train, 
                                                                 epochs = 1000, 
                                                                 batch_size = 512, 
                                                                 shuffle = True,
                                                                 validation_split = 0.2,
                                                                 )

Epoch 1/1000


ValueError: ignored

In [ ]:
classification_model.save_weights('/content/drive/MyDrive/AI-project/Gesture-Recognition/dense_sgdclassification_model')

## 1LayerLSTMADAMDensex3

In [ ]:
opt = tf.keras.optimizers.Adam()
from keras.layers.normalization.batch_normalization import BatchNormalization
from tensorflow.keras.layers import *
classification_model = Sequential([LSTM(1024),
                                   Flatten(),
                                   Dense(512, activation='relu'),
                                   Dense(256, activation='relu'),
                                   Dense(10),
                                   Softmax()])

classification_model.compile(loss = 'categorical_crossentropy', optimizer = opt, metrics = ["accuracy"])


In [ ]:
classification_model_training_history = classification_model.fit(x = x_train, 
                                                                 y = y_train, 
                                                                 epochs = 1000, 
                                                                 batch_size = 512, 
                                                                 shuffle = True,
                                                                 validation_split = 0.2,
                                                                 )

Epoch 1/12
2/2 [==============================] - 2s 2s/step - loss: 1.9814 - accuracy: 0.4050 - val_loss: 2.2110 - val_accuracy: 0.1937
Epoch 2/12
2/2 [==============================] - 2s 2s/step - loss: 1.9455 - accuracy: 0.4300 - val_loss: 2.2144 - val_accuracy: 0.1848
Epoch 3/12
2/2 [==============================] - 2s 2s/step - loss: 1.9245 - accuracy: 0.4700 - val_loss: 2.2162 - val_accuracy: 0.1848
Epoch 4/12
2/2 [==============================] - 2s 2s/step - loss: 1.9102 - accuracy: 0.4650 - val_loss: 2.2144 - val_accuracy: 0.1787
Epoch 5/12
2/2 [==============================] - 2s 2s/step - loss: 1.8868 - accuracy: 0.4900 - val_loss: 2.2108 - val_accuracy: 0.1882
Epoch 6/12
2/2 [==============================] - 2s 2s/step - loss: 1.8724 - accuracy: 0.5000 - val_loss: 2.2086 - val_accuracy: 0.1854
Epoch 7/12
2/2 [==============================] - 2s 2s/step - loss: 1.8606 - accuracy: 0.5200 - val_loss: 2.2037 - val_accuracy: 0.1920
Epoch 8/12
2/2 [=========================

In [ ]:
# classification_model.save_weights('/content/drive/MyDrive/AI-project/Gesture-Recognition/1LayerLSTMADAMDensex3_sgdclassification_model')
classification_model.save_weights('/content/drive/MyDrive/AI-project/Gesture-Recognition/Resnet1LayerLSTMADAMDensex3_sgdclassification_model')

## 1LayerGRU

In [ ]:
opt = tf.keras.optimizers.Adam()
from keras.layers.normalization.batch_normalization import BatchNormalization
from tensorflow.keras.layers import *
classification_model = Sequential([GRU(1024),
                                   Flatten(),
                                   Dense(512, activation='relu'),
                                   Dense(256, activation='relu'),
                                   Dense(10),
                                   Softmax()])

classification_model.compile(loss = 'categorical_crossentropy', optimizer = opt, metrics = ["accuracy"])
classification_model_training_history = classification_model.fit(x = x_train, 
                                                                 y = y_train, 
                                                                 epochs = 30, 
                                                                 batch_size = 512, 
                                                                 shuffle = True,
                                                                 validation_split = 0.2,
                                                                 )


Epoch 1/30


KeyboardInterrupt: ignored

In [ ]:
classification_model.save_weights('/content/drive/MyDrive/AI-project/Gesture-Recognition/1LayerGRUADAMDensex3classification_model')

## MultilayeredLSTM

In [ ]:
opt = tf.keras.optimizers.Adam()
from keras.layers.normalization.batch_normalization import BatchNormalization
from tensorflow.keras.layers import *
classification_model = Sequential([LSTM(1024, return_sequences=True),
                                   LSTM(512, return_sequences=True),
                                   LSTM(10),
                                   Softmax()])

classification_model.compile(loss = 'categorical_crossentropy', optimizer = opt, metrics = ["accuracy"])



In [ ]:
classification_model_training_history = classification_model.fit(x = x_train, 
                                                                 y = y_train, 
                                                                 epochs = 60, 
                                                                 batch_size = 512, 
                                                                 shuffle = True,
                                                                 validation_split = 0.2,
                                                                 )

Epoch 1/60
4/4 [==============================] - 7s 540ms/step - loss: 2.2420 - accuracy: 0.1327 - val_loss: 2.1506 - val_accuracy: 0.2140
Epoch 2/60
4/4 [==============================] - 1s 204ms/step - loss: 1.9692 - accuracy: 0.2998 - val_loss: 1.8849 - val_accuracy: 0.4039
Epoch 3/60
4/4 [==============================] - 1s 203ms/step - loss: 1.7997 - accuracy: 0.3714 - val_loss: 1.6761 - val_accuracy: 0.5590
Epoch 4/60
4/4 [==============================] - 1s 204ms/step - loss: 1.6064 - accuracy: 0.5784 - val_loss: 1.5418 - val_accuracy: 0.6332
Epoch 5/60
4/4 [==============================] - 1s 206ms/step - loss: 1.4732 - accuracy: 0.6832 - val_loss: 1.4896 - val_accuracy: 0.6070
Epoch 6/60
4/4 [==============================] - 1s 205ms/step - loss: 1.3843 - accuracy: 0.6739 - val_loss: 1.3545 - val_accuracy: 0.7118
Epoch 7/60
4/4 [==============================] - 1s 204ms/step - loss: 1.3217 - accuracy: 0.7182 - val_loss: 1.3759 - val_accuracy: 0.6659
Epoch 8/60
4/4 [====

In [ ]:
classification_model.save_weights('/content/drive/MyDrive/AI-project/Gesture-Recognition/MultiLayeredLSTM')

## Conv1DADAM

In [ ]:
opt = tf.keras.optimizers.Adam()
from keras.layers.normalization.batch_normalization import BatchNormalization
from tensorflow.keras.layers import *
classification_model = Sequential([BatchNormalization(),
                                   Conv1D(128, 5, padding='same'),
                                   Flatten(),
                                   Dense(512, activation='relu'),
                                   Dense(256, activation='relu'),
                                   Dense(10),
                                   Softmax()])

classification_model.compile(loss = 'categorical_crossentropy', optimizer = opt, metrics = ["accuracy"])

classification_model_training_history = classification_model.fit(x = x_train, 
                                                                 y = y_train, 
                                                                 epochs = 1000, 
                                                                 batch_size = 512, 
                                                                 shuffle = True,
                                                                 validation_split = 0.2,
                                                                 )


Epoch 1/1000
4/4 [==============================] - 11s 2s/step - loss: 1.4963 - accuracy: 0.5030 - val_loss: 1.7795 - val_accuracy: 0.5109
Epoch 2/1000
4/4 [==============================] - 10s 2s/step - loss: 0.4016 - accuracy: 0.8744 - val_loss: 1.2553 - val_accuracy: 0.6397
Epoch 3/1000
4/4 [==============================] - 10s 2s/step - loss: 0.1900 - accuracy: 0.9323 - val_loss: 1.0812 - val_accuracy: 0.6965
Epoch 4/1000
4/4 [==============================] - 10s 2s/step - loss: 0.0737 - accuracy: 0.9749 - val_loss: 0.9717 - val_accuracy: 0.7533
Epoch 5/1000
4/4 [==============================] - 12s 3s/step - loss: 0.0223 - accuracy: 0.9940 - val_loss: 1.4957 - val_accuracy: 0.6616
Epoch 6/1000
4/4 [==============================] - 10s 2s/step - loss: 0.0175 - accuracy: 0.9967 - val_loss: 1.4458 - val_accuracy: 0.6921
Epoch 7/1000
4/4 [==============================] - 10s 2s/step - loss: 0.0046 - accuracy: 1.0000 - val_loss: 1.6379 - val_accuracy: 0.6659
Epoch 8/1000
4/4 [==

KeyboardInterrupt: ignored

In [ ]:
classification_model.save_weights('/content/drive/MyDrive/AI-project/Gesture-Recognition/1LayerConv1DADAMDensex3_sgdclassification_model')

## Convolution-Only-With-SGD

In [ ]:
opt = tf.keras.optimizers.SGD()
from keras.layers.normalization.batch_normalization import BatchNormalization
from tensorflow.keras.layers import *
classification_model = Sequential([BatchNormalization(),
                                   Conv1D(512, 7, padding='same'),
                                   Conv1D(512, 7, padding='same', strides=2),
                                   Conv1D(256, 5, padding='same'),
                                   Conv1D(256, 5, padding='same', strides=2),
                                   Conv1D(128, 3, padding='same'),
                                   Conv1D(128, 3, padding='same', strides=2),
                                   Flatten(),
                                   Dense(10),
                                   Softmax()])

classification_model.compile(loss = 'categorical_crossentropy', optimizer = opt, metrics = ["accuracy"])

In [ ]:

classification_model_training_history = classification_model.fit(x = x_train, 
                                                                 y = y_train, 
                                                                 epochs = 100, 
                                                                 batch_size = 512, 
                                                                 shuffle = True,
                                                                 validation_split = 0.2,
                                                                 )


Epoch 1/100


In [ ]:
classification_model.save_weights('/content/drive/MyDrive/AI-project/Gesture-Recognition/FulyConvolutionSGD_sgdclassification_model')

In [ ]:
classification_model.load_weights('/content/drive/MyDrive/AI-project/Gesture-Recognition/FulyConvolutionSGD_sgdclassification_model')

## Convolution-Only-ADAM

In [ ]:
opt = tf.keras.optimizers.Adam()
from keras.layers.normalization.batch_normalization import BatchNormalization
from tensorflow.keras.layers import *
classification_model = Sequential([BatchNormalization(),
                                   Conv1D(512, 7, padding='same'),
                                   Conv1D(512, 7, padding='same', strides=2),
                                   Conv1D(256, 5, padding='same'),
                                   Conv1D(256, 5, padding='same', strides=2),
                                   Conv1D(128, 3, padding='same'),
                                   Conv1D(128, 3, padding='same', strides=2),
                                   Flatten(),
                                   Dense(10),
                                   Softmax()])

classification_model.compile(loss = 'categorical_crossentropy', optimizer = opt, metrics = ["accuracy"])

classification_model_training_history = classification_model.fit(x = x_train, 
                                                                 y = y_train, 
                                                                 epochs = 100, 
                                                                 batch_size = 512, 
                                                                 shuffle = True,
                                                                 validation_split = 0.2,
                                                                 )


Epoch 1/100
4/4 [==============================] - 5s 728ms/step - loss: 8.4012 - accuracy: 0.3654 - val_loss: 18.3911 - val_accuracy: 0.3384
Epoch 2/100
4/4 [==============================] - 1s 134ms/step - loss: 3.5967 - accuracy: 0.6794 - val_loss: 5.5054 - val_accuracy: 0.4127
Epoch 3/100
4/4 [==============================] - 1s 131ms/step - loss: 1.3773 - accuracy: 0.8198 - val_loss: 7.3203 - val_accuracy: 0.5109
Epoch 4/100
4/4 [==============================] - 1s 131ms/step - loss: 1.0732 - accuracy: 0.8553 - val_loss: 3.5378 - val_accuracy: 0.5830
Epoch 5/100
4/4 [==============================] - 1s 126ms/step - loss: 0.4328 - accuracy: 0.9143 - val_loss: 2.7693 - val_accuracy: 0.6507
Epoch 6/100
4/4 [==============================] - 1s 130ms/step - loss: 0.2055 - accuracy: 0.9487 - val_loss: 2.4676 - val_accuracy: 0.6681
Epoch 7/100
4/4 [==============================] - 1s 134ms/step - loss: 0.0803 - accuracy: 0.9792 - val_loss: 2.5631 - val_accuracy: 0.6965
Epoch 8/100


KeyboardInterrupt: ignored

In [ ]:
classification_model.save_weights('/content/drive/MyDrive/AI-project/Gesture-Recognition/FulyConvolutionAdamclassification_model')

#Evaluation Model

In [ ]:
x_test = np.load('/content/drive/MyDrive/AI-project/Gesture-Recognition/features/video_features_densenet_2300_testset.npy')
y_test = np.load('/content/drive/MyDrive/AI-project/Gesture-Recognition/features/labels_densenet_2300_testset.npy')

In [ ]:
y_test = tf.one_hot(y_test, 10)

In [ ]:
x_test.shape

(768, 16, 1920)

## Evaluate Function

In [ ]:
from sklearn.metrics import classification_report
import numpy as np

def evaluate(model, x, y):
  y = np.argmax(y, axis=1) # Convert one-hot to index
  y_pred = model.predict(x)
  y_pred = np.argmax(y_pred, axis=1)
  print(classification_report(y, y_pred))

## Dense SGD Eval

In [ ]:
opt = tf.keras.optimizers.SGD()
from keras.layers.normalization.batch_normalization import BatchNormalization
from tensorflow.keras.layers import *
classification_model = Sequential([
                                   Flatten(),
                                   Dense(512, activation='relu'),
                                   Dense(256, activation='relu'),
                                   Dense(10),
                                   Softmax()])

classification_model.compile(loss = 'categorical_crossentropy', optimizer = opt, metrics = ["accuracy"])
classification_model.load_weights('/content/drive/MyDrive/AI-project/Gesture-Recognition/dense_sgdclassification_model')
classification_model.evaluate(x_test, y_test)

24/24 [==============================] - 0s 4ms/step - loss: 0.6133 - accuracy: 0.7995


[0.6132729649543762, 0.7994791865348816]

In [ ]:
evaluate(classification_model, x_test, y_test)

              precision    recall  f1-score   support

           0       0.85      0.87      0.86        83
           1       0.95      0.88      0.92        86
           2       0.68      0.73      0.71        82
           3       0.92      0.81      0.86        84
           4       0.51      0.76      0.61        25
           5       0.78      0.88      0.83        86
           6       0.77      0.67      0.72        88
           7       0.78      0.68      0.73        82
           8       0.86      0.81      0.83        80
           9       0.78      0.88      0.82        72

    accuracy                           0.80       768
   macro avg       0.79      0.80      0.79       768
weighted avg       0.81      0.80      0.80       768



## Dense Adam Eval

In [ ]:
opt = tf.keras.optimizers.Adam()
from keras.layers.normalization.batch_normalization import BatchNormalization
from tensorflow.keras.layers import *
classification_model = Sequential([
                                   Flatten(),
                                   Dense(512, activation='relu'),
                                   Dense(256, activation='relu'),
                                   Dense(10),
                                   Softmax()])

classification_model.compile(loss = 'categorical_crossentropy', optimizer = opt, metrics = ["accuracy"])
classification_model.load_weights('/content/drive/MyDrive/AI-project/Gesture-Recognition/dense_adamclassification_model')
classification_model.evaluate(x_test, y_test)

24/24 [==============================] - 0s 4ms/step - loss: 0.5950 - accuracy: 0.7982


[0.5949971079826355, 0.7981770634651184]

In [ ]:
evaluate(classification_model, x_test, y_test)

              precision    recall  f1-score   support

           0       0.89      0.86      0.87        83
           1       0.96      0.91      0.93        86
           2       0.64      0.66      0.65        82
           3       0.89      0.85      0.87        84
           4       0.50      0.72      0.59        25
           5       0.80      0.93      0.86        86
           6       0.68      0.66      0.67        88
           7       0.79      0.70      0.74        82
           8       0.88      0.80      0.84        80
           9       0.81      0.86      0.83        72

    accuracy                           0.80       768
   macro avg       0.78      0.79      0.79       768
weighted avg       0.80      0.80      0.80       768



## 1LSTM Eval

In [ ]:
opt = tf.keras.optimizers.Adam()
from keras.layers.normalization.batch_normalization import BatchNormalization
from tensorflow.keras.layers import *
classification_model = Sequential([LSTM(1024),
                                   Flatten(),
                                   Dense(512, activation='relu'),
                                   Dense(256, activation='relu'),
                                   Dense(10),
                                   Softmax()])

classification_model.compile(loss = 'categorical_crossentropy', optimizer = opt, metrics = ["accuracy"])
classification_model.load_weights('/content/drive/MyDrive/AI-project/Gesture-Recognition/1LayerLSTMADAMDensex3_sgdclassification_model')
classification_model.evaluate(x_test, y_test)

NameError: ignored

In [ ]:
evaluate(classification_model, x_test, y_test)

              precision    recall  f1-score   support

           0       0.86      0.84      0.85        83
           1       0.95      0.93      0.94        86
           2       0.64      0.74      0.69        82
           3       0.91      0.87      0.89        84
           4       0.60      0.84      0.70        25
           5       0.80      0.86      0.83        86
           6       0.79      0.70      0.75        88
           7       0.79      0.70      0.74        82
           8       0.87      0.85      0.86        80
           9       0.85      0.83      0.84        72

    accuracy                           0.82       768
   macro avg       0.81      0.82      0.81       768
weighted avg       0.82      0.82      0.82       768



## 1LayerGRU

In [ ]:
opt = tf.keras.optimizers.Adam()
from keras.layers.normalization.batch_normalization import BatchNormalization
from tensorflow.keras.layers import *
classification_model = Sequential([GRU(1024),
                                   Flatten(),
                                   Dense(512, activation='relu'),
                                   Dense(256, activation='relu'),
                                   Dense(10),
                                   Softmax()])

classification_model.compile(loss = 'categorical_crossentropy', optimizer = opt, metrics = ["accuracy"])
classification_model.load_weights('/content/drive/MyDrive/AI-project/Gesture-Recognition/1LayerGRUADAMDensex3classification_model')
classification_model.evaluate(x_test, y_test)

24/24 [==============================] - 1s 10ms/step - loss: 0.9710 - accuracy: 0.8125


[0.9710409641265869, 0.8125]

In [ ]:
evaluate(classification_model, x_test, y_test)

              precision    recall  f1-score   support

           0       0.85      0.88      0.86        83
           1       0.93      0.93      0.93        86
           2       0.60      0.76      0.67        82
           3       0.82      0.89      0.86        84
           4       0.76      0.76      0.76        25
           5       0.81      0.87      0.84        86
           6       0.80      0.64      0.71        88
           7       0.78      0.71      0.74        82
           8       0.92      0.81      0.86        80
           9       0.88      0.85      0.87        72

    accuracy                           0.81       768
   macro avg       0.82      0.81      0.81       768
weighted avg       0.82      0.81      0.81       768



## 1LayerConv1DAdam

In [ ]:
opt = tf.keras.optimizers.Adam()
from keras.layers.normalization.batch_normalization import BatchNormalization
from tensorflow.keras.layers import *
classification_model = Sequential([BatchNormalization(),
                                   Conv1D(128, 5, padding='same'),
                                   Flatten(),
                                   Dense(512, activation='relu'),
                                   Dense(256, activation='relu'),
                                   Dense(10),
                                   Softmax()])

classification_model.compile(loss = 'categorical_crossentropy', optimizer = opt, metrics = ["accuracy"])
classification_model.load_weights('/content/drive/MyDrive/AI-project/Gesture-Recognition/1LayerConv1DADAMDensex3_sgdclassification_model')
classification_model.evaluate(x_test, y_test)

24/24 [==============================] - 1s 5ms/step - loss: 1.2302 - accuracy: 0.7969


[1.230242133140564, 0.796875]

In [ ]:
evaluate(classification_model, x_test, y_test)

              precision    recall  f1-score   support

           0       0.85      0.81      0.83        83
           1       0.97      0.88      0.93        86
           2       0.66      0.77      0.71        82
           3       0.88      0.86      0.87        84
           4       0.54      0.80      0.65        25
           5       0.82      0.87      0.84        86
           6       0.66      0.66      0.66        88
           7       0.77      0.67      0.72        82
           8       0.93      0.82      0.87        80
           9       0.81      0.83      0.82        72

    accuracy                           0.80       768
   macro avg       0.79      0.80      0.79       768
weighted avg       0.81      0.80      0.80       768



## MultiLayeredLSTM

In [ ]:
opt = tf.keras.optimizers.Adam()
from keras.layers.normalization.batch_normalization import BatchNormalization
from tensorflow.keras.layers import *
classification_model = Sequential([LSTM(1024, return_sequences=True),
                                   LSTM(512, return_sequences=True),
                                   LSTM(10),
                                   Softmax()])

classification_model.compile(loss = 'categorical_crossentropy', optimizer = opt, metrics = ["accuracy"])
classification_model.load_weights('/content/drive/MyDrive/AI-project/Gesture-Recognition/MultiLayeredLSTM')
classification_model.evaluate(x_test, y_test)

24/24 [==============================] - 14s 14ms/step - loss: 1.1898 - accuracy: 0.7747


[1.1897894144058228, 0.7747395634651184]

In [ ]:
evaluate(classification_model, x_test, y_test)

              precision    recall  f1-score   support

           0       0.85      0.84      0.85        83
           1       0.95      0.88      0.92        86
           2       0.50      0.85      0.63        82
           3       0.96      0.76      0.85        84
           4       0.59      0.76      0.67        25
           5       0.82      0.86      0.84        86
           6       0.88      0.40      0.55        88
           7       0.72      0.71      0.72        82
           8       0.88      0.84      0.86        80
           9       0.78      0.86      0.82        72

    accuracy                           0.77       768
   macro avg       0.79      0.78      0.77       768
weighted avg       0.81      0.77      0.77       768



## ConvolutionOnlySGD

In [ ]:
opt = tf.keras.optimizers.SGD()
from keras.layers.normalization.batch_normalization import BatchNormalization
from tensorflow.keras.layers import *
classification_model = Sequential([BatchNormalization(),
                                   Conv1D(512, 7, padding='same'),
                                   Conv1D(512, 7, padding='same', strides=2),
                                   Conv1D(256, 5, padding='same'),
                                   Conv1D(256, 5, padding='same', strides=2),
                                   Conv1D(128, 3, padding='same'),
                                   Conv1D(128, 3, padding='same', strides=2),
                                   Flatten(),
                                   Dense(10),
                                   Softmax()])

classification_model.compile(loss = 'categorical_crossentropy', optimizer = opt, metrics = ["accuracy"])
classification_model.load_weights('/content/drive/MyDrive/AI-project/Gesture-Recognition/FulyConvolutionSGD_sgdclassification_model')
classification_model.evaluate(x_test, y_test)

24/24 [==============================] - 1s 11ms/step - loss: 0.7938 - accuracy: 0.7852


[0.7937566637992859, 0.78515625]

In [ ]:
evaluate(classification_model, x_test, y_test)

              precision    recall  f1-score   support

           0       0.85      0.82      0.83        83
           1       0.93      0.87      0.90        86
           2       0.65      0.65      0.65        82
           3       0.85      0.83      0.84        84
           4       0.59      0.76      0.67        25
           5       0.84      0.88      0.86        86
           6       0.65      0.60      0.62        88
           7       0.75      0.77      0.76        82
           8       0.88      0.82      0.85        80
           9       0.74      0.83      0.78        72

    accuracy                           0.79       768
   macro avg       0.77      0.78      0.78       768
weighted avg       0.79      0.79      0.79       768



## ConvolutionOnlyAdam

In [ ]:
opt = tf.keras.optimizers.Adam()
from keras.layers.normalization.batch_normalization import BatchNormalization
from tensorflow.keras.layers import *
classification_model = Sequential([BatchNormalization(),
                                   Conv1D(512, 7, padding='same'),
                                   Conv1D(512, 7, padding='same', strides=2),
                                   Conv1D(256, 5, padding='same'),
                                   Conv1D(256, 5, padding='same', strides=2),
                                   Conv1D(128, 3, padding='same'),
                                   Conv1D(128, 3, padding='same', strides=2),
                                   Flatten(),
                                   Dense(10),
                                   Softmax()])

classification_model.compile(loss = 'categorical_crossentropy', optimizer = opt, metrics = ["accuracy"])
classification_model.load_weights('/content/drive/MyDrive/AI-project/Gesture-Recognition/FulyConvolutionAdamclassification_model')
classification_model.evaluate(x_test, y_test)

24/24 [==============================] - 1s 11ms/step - loss: 2.5670 - accuracy: 0.7747


[2.566965341567993, 0.7747395634651184]

In [ ]:
evaluate(classification_model, x_test, y_test)

              precision    recall  f1-score   support

           0       0.88      0.82      0.85        83
           1       0.99      0.88      0.93        86
           2       0.62      0.60      0.61        82
           3       0.86      0.83      0.85        84
           4       0.53      0.76      0.62        25
           5       0.79      0.86      0.82        86
           6       0.61      0.61      0.61        88
           7       0.73      0.70      0.71        82
           8       0.80      0.81      0.81        80
           9       0.83      0.88      0.85        72

    accuracy                           0.77       768
   macro avg       0.76      0.78      0.77       768
weighted avg       0.78      0.77      0.78       768



#Predict on a new video

In [ ]:
#!pip install pafy youtube-dl moviepy
import pafy

In [ ]:
def download_youtube_videos(youtube_video_url, output_directory):
    video = pafy.new(youtube_video_url)
    video_best = video.getbest()
    output_file_path = f'{output_directory}/{video.title}.mp4'
    video_best.download(filepath = output_file_path, quiet = True)
    return video.title

In [ ]:
def predict_on_live_video(video_file_path, output_file_path, window_size):
    # Initialize a Deque Object with a fixed size which will be used to implement moving/rolling average functionality.
    predicted_labels_probabilities_deque = deque(maxlen = window_size)
    # Reading the Video File using the VideoCapture Object
    video_reader = cv2.VideoCapture(video_file_path)
    # Getting the width and height of the video 
    original_video_width = int(video_reader.get(cv2.CAP_PROP_FRAME_WIDTH))
    original_video_height = int(video_reader.get(cv2.CAP_PROP_FRAME_HEIGHT))
    # Writing the Overlayed Video Files Using the VideoWriter Object
    video_writer = cv2.VideoWriter(output_file_path, cv2.VideoWriter_fourcc('M', 'P', '4', 'V'), 24, (original_video_width, original_video_height))

    while True: 
        status, frame = video_reader.read() 
        if not status:
            break
        resized_frame = cv2.resize(frame, (IMAGE_HEIGHT, IMAGE_WIDTH))
        normalized_frame = resized_frame / 255

        # Passing the Image Normalized Frame to the model and receiving Predicted Probabilities.
        predicted_labels_probabilities = model.predict(np.expand_dims(normalized_frame, axis = 0))[0]

        # Appending predicted label probabilities to the deque object
        predicted_labels_probabilities_deque.append(predicted_labels_probabilities)

        # Assuring that the Deque is completely filled before starting the averaging process
        if len(predicted_labels_probabilities_deque) == window_size:

            # Converting Predicted Labels Probabilities Deque into Numpy array
            predicted_labels_probabilities_np = np.array(predicted_labels_probabilities_deque)

            # Calculating Average of Predicted Labels Probabilities Column Wise 
            predicted_labels_probabilities_averaged = predicted_labels_probabilities_np.mean(axis = 0)

            # Converting the predicted probabilities into labels by returning the index of the maximum value.
            predicted_label = np.argmax(predicted_labels_probabilities_averaged)

            # Accessing The Class Name using predicted label.
            predicted_class_name = classes_list[predicted_label]
          
            # Overlaying Class Name Text Ontop of the Frame
            cv2.putText(frame, predicted_class_name, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

        # Writing The Frame
        video_writer.write(frame)

        # cv2.imshow('Predicted Frames', frame)
        # key_pressed = cv2.waitKey(10)
        # if key_pressed == ord('q'):
        #     break

    # cv2.destroyAllWindows()

    # Closing the VideoCapture and VideoWriter objects and releasing all resources held by them. 
    video_reader.release()
    video_writer.release()